# Legal Robots: Datenbanken (3)

Da **A.A.** mittlerweile einiges über Datenbanken weiß und ihm dieser Ansatz besser gefällt,
als alle Daten in Dateien zu schreiben, möchte er die Entscheidungen, 
die er von RII heruntergeladen hatte, nun ebenfalls in eine Datenbank schreiben. 

Wir wollen ihm auch dabei behilflich sein.
Dazu öffnen wir zunächst eine Entscheidung und sehen uns an, 
welche Informationen wir strukturiert ablegen können.

In [4]:
import sqlite3

conn = sqlite3.connect('rii.db')
    
cursor = conn.cursor()   
cursor.execute("DROP TABLE IF EXISTS Entscheidungen")
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Entscheidungen ( 
        doknr TEXT PRIMARY KEY,
        ecli TEXT,
        gertyp TEXT,
        gerort TEXT,
        spruchkoerper TEXT,
        entscheidungsdatum TEXT,
        aktenzeichen TEXT,
        doktyp TEXT,
        norm TEXT,
        vorinstanz TEXT,
        identifier TEXT,
        volltext TEXT);''')

conn.commit()
conn.close()

Ein Hilfsroboter soll das Einfügen aller Dokumente in die Datenbank übernehmen:

In [9]:
import sqlite3, datetime, os
from bs4 import BeautifulSoup

class ScrapingHelperRobot:
    
    def __init__(self, name):
        self.name = name
        self.connection = sqlite3.connect('rii.db')
        self.cursor = self.connection.cursor()  
    
    def insert_all_files(self):
        for file in os.listdir("rii/"):
            try:
                filehandle = open("rii/" + file, 'r', encoding='utf8')
                filesoup = BeautifulSoup(filehandle, "xml")
                dokument = filesoup.dokument
            
                values = {
                    "doknr" : dokument.doknr.get_text(),
                    "ecli" : dokument.ecli.get_text(),
                    "gertyp" : dokument.gertyp.get_text(),
                    "gerort" : dokument.gerort.get_text(),
                    "spruchkoerper" : dokument.spruchkoerper.get_text(),
                    "entscheidungsdatum" : dokument.find("entsch-datum").get_text(),
                    "aktenzeichen" : dokument.aktenzeichen.get_text(),
                    "doktyp" : dokument.doktyp.get_text(),
                    "norm" : dokument.norm.get_text(),
                    "vorinstanz" : dokument.vorinstanz.get_text(),
                    "identifier" : dokument.identifier.get_text(),
                    "volltext" : dokument.get_text()
                }

                sql = '''INSERT INTO Entscheidungen 
                    (doknr, ecli, gertyp, gerort, spruchkoerper, entscheidungsdatum, aktenzeichen, doktyp, norm, vorinstanz, identifier, volltext) 
                    VALUES (:doknr, :ecli, :gertyp, :gerort, :spruchkoerper, :entscheidungsdatum, :aktenzeichen, :doktyp, :norm, :vorinstanz, :identifier, :volltext)'''
                self.cursor.execute(sql, values)
            except UnicodeDecodeError:
                pass

        self.connection.commit()
    
    def __del__(self):
        self.connection.close()

___

## Aufgabe 1: Code verstehen

a) Betrachte die Implementierung des `ScrapingHelperRobot` und mache dir bei jedem Befehl klar, was das Programm tut. 
Wo verwenden wir bisher unbekannte Syntaxelemente? 
Was bewirken diese?

b) Ändere den Code so ab, dass zum Testen nur die ersten fünf Elemente verarbeitet werden.
Erzeuge dann einen `ScrapingHelperRobot` mit dem Namen `SHR` und prüfe, ob die Implementierung funktioniert. 
Wenn dies der Fall ist, kannst du `SHR` alle Entscheidungen einlesen lassen.

c) Schau dir zunächst die Daten mit einem SQLite-Browser an (z.B. [DB Browser for SQLite](http://sqlitebrowser.org/)). 
Welche Felder sind ordentlich gepflegt, welche weniger?

d) Führe die folgenden Abfragen aus und mache dir klar, was passiert. 
Spiele mit den Daten aus der Datenbank, um unterschiedliche Einblicke zu gewinnen.
___

Jetzt können wir schon einige interessante Auswertungen vornehmen. 
Dazu nutzen wir nun das **SQL-Magic** von IPython:

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///rii.db

### Zahlenmäßige Auswertungen

In [ ]:
%sql SELECT COUNT (*) FROM Entscheidungen

In [ ]:
%sql SELECT doktyp, COUNT (*) FROM Entscheidungen GROUP BY doktyp

In [ ]:
%%sql
SELECT gertyp, COUNT (*) 
FROM Entscheidungen 
GROUP BY gertyp

> **Hinweis:**

> Es gibt zwei Arten von Magics in IPython, `Line Magics` und `Cell Magics`. 
`Line Magics` haben das Präfix '`%`', `Cell Magics` das Präfix '`%%`'. 
Das SQL-Magic gibt es als `Line Magic` *und* als `Cell Magic`. 
Versuche, anhand der Syntax zu verstehen, worin genau der Unterschied liegt.

In [ ]:
%%sql
SELECT gertyp, spruchkoerper, COUNT (*) 
FROM Entscheidungen 
GROUP BY gertyp, spruchkoerper

In [ ]:
%%sql
SELECT SUBSTR(entscheidungsdatum,1,4) AS Jahr, COUNT (*) 
FROM Entscheidungen 
GROUP BY Jahr

In [ ]:
%%sql
SELECT SUBSTR(entscheidungsdatum,5,2) AS Monat, COUNT (*) 
FROM Entscheidungen 
GROUP BY Monat

### Neueste Entscheidungen

In [ ]:
%%sql
SELECT doknr, ecli, gertyp, gerort, spruchkoerper, entscheidungsdatum, aktenzeichen, 
doktyp, norm, vorinstanz, identifier 
FROM Entscheidungen 
ORDER BY entscheidungsdatum DESC 
LIMIT 3

In [ ]:
%%sql
SELECT gertyp, MAX(entscheidungsdatum) AS neuesteEntscheidungVom 
FROM Entscheidungen 
GROUP BY gertyp

___

## Aufgabe 2: Code weiterentwickeln

a) Das Gericht wird nur mit einer Abkürzung angezeigt und der Gerichtsort ist meistens nicht ordentlich gepflegt. 
Erstelle eine Tabelle `Gerichte`, in die die Abkürzung `gertyp` (*Primary Key*!), `bezeichnung` und `ort` eingetragen werden und fülle sie mit den wichtigsten Daten. 
Du kannst mit den SQL Magics arbeiten.

b) Erstelle eine Abfrage für die fünf neuesten Entscheidungen, wobei als Gericht und Gerichtsort die längere Bezeichnung aus der `Gerichte`-Tabelle und der dort vermerkte Ort anbezeigt werden sollen. 